In [1]:
import json
from apache_beam.options.pipeline_options import PipelineOptions

class CustomOptions(PipelineOptions):
    @classmethod
    def _add_argparse_args(cls, parser):
        parser.add_argument('--subscription_path')
        parser.add_argument('--output_schema')

args = PipelineOptions().view_as(CustomOptions)
args.subscription_path = 'projects/claritas-bigdata-poc/subscriptions/store'
args.output_schema = json.loads("""{"fields": [
    {"name": "order_id", "type": "STRING", "mode": "REQUIRED"},
    {"name": "datetime", "type": "DATETIME", "mode": "REQUIRED"},
    {"name": "ordered_item", "type": "RECORD", "mode": "REPEATED", "fields": [
        {"name": "item_name", "type": "STRING", "mode": "REQUIRED"},
        {"name": "item_id", "type": "STRING", "mode": "REQUIRED"},
        {"name": "category_name", "type": "STRING", "mode": "REQUIRED"},
        {"name": "item_price", "type": "FLOAT", "mode": "REQUIRED"},
        {"name": "item_qty", "type": "INTEGER", "mode": "REQUIRED"}
    ]}
]}""")

In [2]:
import json, ast
import apache_beam as beam

class DecodeByte(beam.DoFn):
    def process(self, element):
        yield ast.literal_eval(json.loads(
            element.decode('utf-8')))

In [ ]:
import os, logging
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io import ReadFromPubSub, WriteToBigQuery

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/meng/work/.GCP_SA/mlee-claritas-bigdata-poc.json'

logging.getLogger().setLevel(logging.INFO)

with beam.Pipeline(options=PipelineOptions(
    project='claritas-bigdata-poc',
    region='asia-southeast1',
    temp_location='gs://claritas-bigdata-poc-temp/',
    runner='DataflowRunner',
    streaming=True,
    save_main_session=True
)) as p:
    
    _ = (p
        | 'Read from Pub/Sub' >> ReadFromPubSub(
            subscription=args.subscription_path)
        | 'Decode bytestring' >> beam.ParDo(DecodeByte())
        | 'Write to BQ' >> WriteToBigQuery(
            table='test.store',
            schema=args.output_schema,
            create_disposition='CREATE_IF_NEEDED',
            write_disposition='WRITE_APPEND')
    )

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


/opt/conda/envs/notebook/lib/python3.8/site-packages/apache_beam/io/gcp/bigquery.py:1683: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/opt/conda/envs/notebook/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmp20_9rnm4', 'apache-beam==2.31.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI: dataflow_python_sdk.tar
INFO:apache_beam.runners.portability.stager:Downloading binary distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/opt/conda/envs/notebook/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmp20_9rnm4', 'apache-beam==2.31.0', '--no-deps', '--only-b